<a href="https://colab.research.google.com/github/Prachii26/modern-ai-unsloth/blob/main/notebooks/Colab3_DPO_REINFORCEMENT_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --upgrade trl

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-zdqwvjav/unsloth_39d132607e9b404090f03ac58783b76e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-zdqwvjav/unsloth_39d132607e9b404090f03ac58783b76e
  Resolved https://github.com/unslothai/unsloth.git to commit 56392a7a4c1bf38154a144854a61b7d2d4dd2d0b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.5/511.5 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.7/276.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 129.1 MB/s eta 0:00:00

In [ ]:
from unsloth import FastLanguageModel, PatchDPOTrainer
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "HuggingFaceTB/SmolLM2-135M-Instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

HuggingFaceTB/SmolLM2-135M-Instruct does not have a padding token! Will use pad_token = <|endoftext|>.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth 2025.11.1 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


In [ ]:
dataset = load_dataset("argilla/distilabel-capybara-dpo-7k-binarized", split="train[:1000]")
dataset = dataset.train_test_split(test_size=0.1)
print(f"Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")
print(dataset['train'][0])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7563 [00:00<?, ? examples/s]

Train: 900, Test: 100
{'source': 'Airoboros', 'conversation': [{'input': "What popular children's book series features a young orphaned wizard attending a magical school called Hogwarts?", 'output': "The Harry Potter series, written by J.K. Rowling, follows the life of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. This beloved book series consists of seven books, beginning with Harry's acceptance into Hogwarts and culminating with his final battle against the dark wizard, Lord Voldemort."}, {'input': 'Describe the educational system and curriculum at Hogwarts School of Witchcraft and Wizardry.', 'output': "Hogwarts School of Witchcraft and Wizardry, as depicted in the Harry Potter series, is a boarding school for young witches and wizards. The school year runs from September 1 to late June, with students returning home for winter and summer holidays.\n\nUpon arrival, first-year stu

In [ ]:
def format_dpo(example):
    prompt = tokenizer.apply_chat_template(example["chosen"][:-1], tokenize=False, add_generation_prompt=True)
    chosen = example["chosen"][-1]["content"]
    rejected = example["rejected"][-1]["content"]
    return {"prompt": prompt, "chosen": chosen, "rejected": rejected}

dataset = dataset.map(format_dpo)
print(dataset['train'][0])

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'source': 'Airoboros', 'conversation': [{'input': "What popular children's book series features a young orphaned wizard attending a magical school called Hogwarts?", 'output': "The Harry Potter series, written by J.K. Rowling, follows the life of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. This beloved book series consists of seven books, beginning with Harry's acceptance into Hogwarts and culminating with his final battle against the dark wizard, Lord Voldemort."}, {'input': 'Describe the educational system and curriculum at Hogwarts School of Witchcraft and Wizardry.', 'output': "Hogwarts School of Witchcraft and Wizardry, as depicted in the Harry Potter series, is a boarding school for young witches and wizards. The school year runs from September 1 to late June, with students returning home for winter and summer holidays.\n\nUpon arrival, first-year students are sorted into 

In [ ]:
PatchDPOTrainer()

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 100,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
    beta = 0.1,
    train_dataset = dataset["train"],
    tokenizer = tokenizer,
    max_length = 1024,
    max_prompt_length = 512,
)

Extracting prompt in train dataset (num_proc=6):   0%|          | 0/900 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=6):   0%|          | 0/900 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=6):   0%|          | 0/900 [00:00<?, ? examples/s]

In [ ]:
dpo_trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 900 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,399,488 (3.50% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: prachi-gupta01 (prachi-gupta01-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
10,0.690900,0.001516,-0.003079,0.475000,0.004595,-665.072632,-835.705200,4.470216,5.156035,0,0,0
20,0.680900,-0.037108,-0.064284,0.612500,0.027176,-775.816528,-920.914673,5.636608,5.516665,No Log,No Log,No Log
30,0.648100,-0.084834,-0.188045,0.787500,0.103211,-702.405090,-982.314636,5.158022,5.716948,No Log,No Log,No Log
40,0.630200,-0.172452,-0.336498,0.737500,0.164046,-679.671692,-908.978516,4.353178,5.057672,No Log,No Log,No Log
50,0.635000,-0.296853,-0.464086,0.650000,0.167232,-762.294800,-912.931335,4.270198,4.762456,No Log,No Log,No Log
60,0.594500,-0.318678,-0.586312,0.700000,0.267635,-780.579956,-1007.041138,4.352482,4.576245,No Log,No Log,No Log
70,0.612800,-0.326747,-0.597307,0.700000,0.270560,-734.366943,-890.557251,4.999740,5.113968,No Log,No Log,No Log
80,0.652500,-0.392961,-0.588241,0.612500,0.195280,-742.022034,-923.408813,4.095695,4.553823,No Log,No Log,No Log
90,0.585000,-0.393395,-0.762964,0.737500,0.369569,-749.316589,-942.307312,4.514369,4.782531,No Log,No Log,No Log
100,0.606200,-0.411725,-0.696379,0.637500,0.284655,-763.979004,-974.649902,4.450908,4.701705,No Log,No Log,No Log


TrainOutput(global_step=100, training_loss=0.6335921287536621, metrics={'train_runtime': 1115.3548, 'train_samples_per_second': 0.717, 'train_steps_per_second': 0.09, 'total_flos': 0.0, 'train_loss': 0.6335921287536621, 'epoch': 0.8888888888888888})

In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "What is the capital of France?"}
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=128, temperature=0.7)
print(tokenizer.decode(outputs[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant
The capital of France is Paris. It is a vibrant city known for its historical landmarks, cultural significance, and cultural diversity. Paris is a city that is steeped in history, with its iconic landmarks such as the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and Arc de Triomphe, and the historic center of the city, with its charming streets and historic neighborhoods.

Paris is also home to many other important historical sites, including the Arc de Triomphe, the Arc de Triomphe, and the Arc de Triomphe, and the Louvre, the Louvre, and the Mus


In [ ]:
model.save_pretrained("smollm2_dpo")
tokenizer.save_pretrained("smollm2_dpo")
print("✅ DPO model saved!")

✅ DPO model saved!
